### Libraries

In [1]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
# from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [3]:
def built_circuit(pauli_string='x', error=0.1, approximated = True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(list(range(len(controls)+1)))
    
    if approximated:
        MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
        Ldmcu.ldmcu(circ, pauli_matrix_dagger, controls, target)
        
    # circ.measure_all()
    
    return circ

In [4]:
approx_circuit = built_circuit('x', 0.1)
print(approx_circuit)

            ┌───┐┌──────────────┐
controls_0: ┤ X ├┤0             ├
            ├───┤│              │
controls_1: ┤ X ├┤1             ├
            ├───┤│              │
controls_2: ┤ X ├┤2             ├
            ├───┤│              │
controls_3: ┤ X ├┤3 Ldmcuapprox ├
            ├───┤│              │
controls_4: ┤ X ├┤4             ├
            ├───┤│              │
controls_5: ┤ X ├┤5             ├
            ├───┤│              │
    target: ┤ X ├┤6             ├
            └───┘└──────────────┘


In [5]:
real_circuit = built_circuit('x', 0.1, False)
print(real_circuit)

            ┌───┐┌────────┐
controls_0: ┤ X ├┤0       ├
            ├───┤│        │
controls_1: ┤ X ├┤1       ├
            ├───┤│        │
controls_2: ┤ X ├┤2       ├
            ├───┤│        │
controls_3: ┤ X ├┤3 Ldmcu ├
            ├───┤│        │
controls_4: ┤ X ├┤4       ├
            ├───┤│        │
controls_5: ┤ X ├┤5       ├
            ├───┤│        │
    target: ┤ X ├┤6       ├
            └───┘└────────┘


In [6]:
from qiskit.quantum_info import DensityMatrix
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

In [7]:
sim = AerSimulator
AerSimulator.available_methods(sim)

In [8]:
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity

def density_matrix(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size)
    noise_model = NoiseModel.from_backend(backend)
    
    simulator = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circuit, simulator)
    
    # job = simulator.run(transpiled_circuit)
    # result = job.result()
    # d_m = result.get_unitary(transpiled_circuit)

    d_m = DensityMatrix(transpiled_circuit)
    return d_m

In [9]:
rc_density_matrix = density_matrix(real_circuit)

In [10]:
ac_density_matrix = density_matrix(approx_circuit)

### Adicionar medida de fidelidade.

In [63]:
from qiskit.quantum_info import state_fidelity